# 🦾  Treat it as a project.
Yeah. I saw an introduction and this dataset looks amazing. This time, we are going to be dealing with a **new** dataset if the **human** activity.

This will be a new kind of problem that we will be trying to solve. Here, the dataset includes the "measurements" taken from the smart watch ⌚ from multiple people throughout the day.

> The goal is to classify **whether** a person is *sitting*, *dancing*, *sleeping* or whatever! Or just bathing in his bathtub. There will be 6 classes to classify. Hence, it is a **multiclass-classification** problem. But, with the time-series.


There are `2` sensors are used for this dataset: 
1. Gyroscope
2. Accelerometer

Each of them returns `3` separate channels because we are living in the 3D space.

___
Dataset metadata:
- There are 2 sensors,.
- Each will return 3 channels.
- Hence, we will be having `3` + `3` = **`6`** features.
- Their names are ***"acceleration"*** & ***"angular velocity"*** returned by accelerometer and gyroscope respectively.
- BUTT: There are **2** types of acceleration:
    1. Total acceleration
    2. Body acceleration = Total acceleration - gravity
- Hence, we have `3` + `3` + `3` = **`9`** final features in the dataset.

About the time-series:

- Each measurement was sampled at `50Hz` = 50 measurements/sec
- Taken at the intervals of `2.56` seconds.
- Thus, we have 50 * 2.56 = `128` measurements for each time-series.
- Meaning, **the leanght** of time series corresponding to **each** activity is 128.

Some pre-processing is done already:

- Noise removal
- Low pass filtering (smoothing - EWMA *if you remember* 😉)
- Scaling [-1, 1]

Hence, the time-series is not in a raw version.

      +--------+
      /        /|
     /    T   / |
    +--------+  |
    |        | D|
    |   N    |  +
    |        | /
    |        |/
    +--------+

Where:
- N: Number of observations
- T: Features
- D: What makes the feature? (128 observations)

Till now, this is the visualization of the data till now. And what about **Y**? It is the simple `1D` data encoded in {0, 1, 2, 3, 4, 5, 6}

#### 🤔 Does this look... a bit strange?
Yes, because till now... we have only worked with the **2D** data. Or just NxT data. Now are are dealing with **Multivariate** data.

Before this, we had **Univariate** data, thus we were able to store them in the 2D table. Here still a 2D table will be there, but the data itself has to be treated *differently*.

> Here is the [link](https://www.kaggle.com/datasets/uciml/human-activity-recognition-with-smartphones) to the dataset. Have a look if there is any confusion... or anything.

## 🛣 Side track

**A multi-variate jargon** <br>
So, till now we have seen the 2D data like...

#### 👉 Before
<img src="../images/univariate.jpg" width=300 height=300>

Where, <br>
- **N** is the number of observations.
- **T** is the number of features.

Such a simple and beautiful thing. <br>
____

#### 👉 Now
<img src="../images/multivariate.jpg" width=300 height=300>

Where, <br>
- **D** is the number of different perspectives to observe the data.

### 🐼 But in pandas...
We can show that like this.

In [1]:
import pandas as pd
import numpy as np

In [2]:
columns = pd.MultiIndex.from_product([["Machine 1", "Machine 2", "Machine 3"], 
                                      ["Height", "Weight"]])

In [3]:
np.random.seed(42)
df = pd.DataFrame(np.random.randint(-10, 10, (5, 3 * 2)), columns=columns)
df = pd.concat([df, pd.Series(
                np.random.choice(["M", "F"], 5),
                name=("", "Gender"))], 
          axis=1)

df.columns.names = ["D →", "T →"]
df.index.name = "N ↓"
df

D → Machine 1        Machine 2        Machine 3              
T →    Height Weight    Height Weight    Height Weight Gender
N ↓                                                          
0          -4      9         4      0        -3     -4      M
1           8      0         0     -7        -3     -8      M
2          -9      1        -5     -9       -10      1      M
3           1      6        -1      5         4      4      M
4           8      1         9     -8        -6      8      F

Yeah. This is it. This is the `N x T x D` shape.

*If you want to see some crazy things...* look below.

The data is like:

In [4]:
multivariate = np.arange(27).reshape(3, 3, 3)
multivariate

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],

       [[ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17]],

       [[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]]])

In [5]:
# N x T x D
multivariate.shape

(3, 3, 3)

To convert **that** into the dataframe style...

#### Option 1

In [6]:
# a chain operation
multivariate.reshape(3, 9, order="F").reshape(9, 3, order="F").reshape(3, 9)

array([[ 0,  1,  2,  9, 10, 11, 18, 19, 20],
       [ 3,  4,  5, 12, 13, 14, 21, 22, 23],
       [ 6,  7,  8, 15, 16, 17, 24, 25, 26]])

#### Option 2

In [7]:
# a loop operation
x = multivariate[0, :].copy()
for i in range(1, multivariate.shape[-1]): # for shape D
    x = np.c_[x, multivariate[i, :]]

In [8]:
x

array([[ 0,  1,  2,  9, 10, 11, 18, 19, 20],
       [ 3,  4,  5, 12, 13, 14, 21, 22, 23],
       [ 6,  7,  8, 15, 16, 17, 24, 25, 26]])

#### 🛣 Back to the track

Now we know **how the data is represented** we would love to see **what shouldn't be done.**

## 🧷 What we did.

1. We had `N x T x D` shape of data.
2. We converted that into `N x TD` data so that can be treated as 2D

After that, the **naive approach** would be to **make the simple NN as we usually do**... 

```python
input_layer = Input(shape=(TD,))
hidden_layer = Dense(...)
...
model = Model(...)
```

Simple right? <br>
But that's **NOT** ideal. Because that means that: ***each dimension is treated equally.***

Since this time-series has a special structure, we would need to take the advantage of that. And also our neural network is so flexible that it can handle this *multivariate* or *multi-input* kind of inputs. We just need to tweak some parameters. 

### ⚒ So we will build multi "tailed" network

<img src="../images/multivariate-nn.png" width=400 height=500>

> 🤔 But...? Why multi **tailed** and not **multi-headed**? <br> <br> 😀 The reason is that, the final layer (output layer) is called the "head" of the neural net. Thus, the inputs are the tails.

And **yes**. The final layer (output layer) can have the multiple layers as well! Which is multi-layer-output.

The code will be like:
```python
# multiple inputes
input_layer1 = Input(shape(T,))
input_layer2 = Input(shape(T,))
input_layer3 = Input(shape(T,))

# hidden layer 1 for each inputs
hidden_layer1_1 = Dense(32, activation="relu")(input_layer1)
hidden_layer1_2 = Dense(32, activation="relu")(input_layer2)
hidden_layer1_3 = Dense(32, activation="relu")(input_layer3)

# this will let us to streamline all seperate layers into a single - sequential layer
full_layer = Concatenate()([hidden_layer1_1, hidden_layer1_2, hidden_layer1_3])

# the final output | K = number of classes (so the nodes)
output_layer = Dense(K)(full_layer)

# the model
model = Model([input_layer1, input_layer2, input_layer3], output_layer)
```

Yes, the code certainly feels like **training multiple seperate neural networks** for each layer... but that is just in the starting portion. When we concatenate, things are not like the *seperate nets*. 

In [12]:
multivariate

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],

       [[ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17]],

       [[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]]])

In [11]:
multivariate[:, :, 0]

array([[ 0,  3,  6],
       [ 9, 12, 15],
       [18, 21, 24]])